# Compare deepcoadds and Objects

 work with Weakly_2024_07
- use jupyter kernel LSST
- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/12/23
- update : 2024/02/24


In [ ]:
! eups list -s | grep LOCAL

In [ ]:
! eups list -s lsst_distrib

In [ ]:
import lsst.daf.butler as dafButler
import lsst.daf.base as dafBase

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm 
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.dates as mdates
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.geom as geom
import pprint

In [ ]:
#Set the matplotlib plot color table.
plt.style.use('tableau-colorblind10')

In [ ]:
# Let us also set `lsst.afw.display` to use the `matplotlib` backend:
afwDisplay.setDefaultBackend('matplotlib')
plt.rcParams['figure.figsize'] = (6.0, 6.0)

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

## Config

### Butler repo and collection

In [ ]:
#repo =  "/sdf/group/rubin/repo/main"
repo = "/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

### Give the collection

In [ ]:
#my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553"
my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871"

### input files

- object file
- tract and patch

In [ ]:
!ls ../data/202402

In [ ]:
file_obs = "../data/202402/objectTable_202402.csv"
file_tractpatch = "../data/202402/Latiss_tractspatchs_202402.pickle"

### DataId

- Tract 5615, patch 294 ou 313
- tract 5634, patch 72,73, 91, 92

In [ ]:
tract = 5615
patch = 294
band = 'r'
skymap = 'latiss_v1'

In [ ]:
the_dataId = {'skymap':skymap,'band': band, 'tract': tract, 'patch': patch}

In [ ]:
the_title = f"Auxtel deepCoadd : band={band}, tract={tract}, patch={patch}" 

## Init

In [ ]:
with open(file_tractpatch , 'rb') as handle:
    tracts_patchs_dict = pickle.load(handle)

In [ ]:
tracts_patchs_dict

In [ ]:
df = pd.read_csv(file_obs,index_col=0) 

In [ ]:
selection1 =  (df["tract"] == tract) & (df["patch"] == patch)
selection2 = df["refExtendedness"] == False
selection3 = df["detect_isPrimary"] == True

In [ ]:
df_sel = df[selection1 & selection2 & selection3]

In [ ]:
df_sel["g_cModelMag"] = df_sel["g_cModelFlux"].map(convert_fluxtomag)
df_sel["r_cModelMag"] = df_sel["r_cModelFlux"].map(convert_fluxtomag)
df_sel["i_cModelMag"] = df_sel["i_cModelFlux"].map(convert_fluxtomag)

## Plot magnitudes No Selection cut

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(14,4))

df_sel['g_cModelMag'].plot(kind="hist",bins=50,range=(5,30),facecolor="g",logy=True,grid=True,title="g_cModelMag",ax=ax1)
df_sel['r_cModelMag'].plot(kind="hist",bins=50,range=(5,30),facecolor="r",logy=True,grid=True,title="r_cModelMag",ax=ax2)
df_sel['i_cModelMag'].plot(kind="hist",bins=50,range=(5,30),facecolor="grey",logy=True,grid=True,title="i_cModelMag",ax=ax3)
plt.tight_layout()
plt.suptitle(the_title)

### Plot deep Coadds without selection

In [ ]:
pos_obj = df_sel[["x","y","g_ra","r_ra","i_ra", "g_dec","r_dec","i_dec" ]]
pos_obj

In [ ]:
my_deepCoadd = butler.get('deepCoadd', dataId=the_dataId,collections=my_collection)
my_deepCoadd_inputs = butler.get("deepCoadd.coaddInputs",the_dataId,collections=my_collection)   

In [ ]:
my_deepCoadd_inputs.visits.asAstropy()

In [ ]:
# Plot the calexp we just retrieved
plt.figure()
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(my_deepCoadd.image,title=the_title )

with afw_display.Buffering():
    for index, row in pos_obj.iterrows():
        x = row["x"]
        y = row["y"]
        afw_display.dot('o', x , y, size=50, ctype='yellow')

plt.gca().axis('on')

#plt.scatter(pos_obj.x.values,pos_obj.y.values,marker="o",color="y")

## Plot magnitude by applying Selection cut

In [ ]:
selection4 = df_sel["r_cModelMag"] < 17.0
df_sel = df_sel[selection4]

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(14,4))

df_sel['g_cModelMag'].plot(kind="hist",bins=50,range=(5,30),facecolor="g",logy=True,grid=True,title="g_cModelMag",ax=ax1)
df_sel['r_cModelMag'].plot(kind="hist",bins=50,range=(5,30),facecolor="r",logy=True,grid=True,title="r_cModelMag",ax=ax2)
df_sel['i_cModelMag'].plot(kind="hist",bins=50,range=(5,30),facecolor="grey",logy=True,grid=True,title="i_cModelMag",ax=ax3)
plt.tight_layout()
plt.suptitle(the_title)

## Position

In [ ]:
#colnames = list(df_sel.columns)
#for col in colnames:
#    if "ra" in col:
#        print(col)

In [ ]:
pos_obj = df_sel[["x","y","g_ra","r_ra","i_ra", "g_dec","r_dec","i_dec" ]]
pos_obj

## deepCoadd

In [ ]:
my_deepCoadd = butler.get('deepCoadd', dataId=the_dataId,collections=my_collection)
my_deepCoadd_inputs = butler.get("deepCoadd.coaddInputs",the_dataId,collections=my_collection)   

In [ ]:
my_deepCoadd_inputs.visits.asAstropy()

## Compare object position with deepcoadds

In [ ]:
# Plot the calexp we just retrieved
plt.figure()
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(my_deepCoadd.image,title=the_title )

with afw_display.Buffering():
    for index, row in pos_obj.iterrows():
        x = row["x"]
        y = row["y"]
        afw_display.dot('o', x , y, size=50, ctype='yellow')

plt.gca().axis('on')

#plt.scatter(pos_obj.x.values,pos_obj.y.values,marker="o",color="y")

In [ ]:
# Plot the calexp we just retrieved
plt.figure(figsize=(12,12))
afw_display = afwDisplay.Display(figsize=(12,12))
afw_display.scale('asinh', 'zscale')
afw_display.mtv(my_deepCoadd.maskedImage.image,title=the_title+" masked")

with afw_display.Buffering():
    for index, row in pos_obj.iterrows():
        x = row["x"]
        y = row["y"]
        afw_display.dot('o', x , y, size=50, ctype='yellow')


plt.gca().axis('on')

## Histo

In [ ]:
fig=plt.figure(figsize=(4,4))
ax=fig.add_subplot(1,1,1)
ax.hist(my_deepCoadd.image.array.flatten(),bins=100,range=(-50,1000),alpha=0.5,facecolor='blue')
ax.hist(my_deepCoadd.maskedImage.image.array.flatten(),bins=100,range=(-50,1000),alpha=0.5,facecolor="yellow")
ax.set_yscale('log')
ax.grid()
ax.set_title(the_title)